In [1]:
!ls /tmp/finn_dev_mirza

end2end_quartznet_export_dev.onnx	    end2end_quartznet_tidy.onnx
end2end_quartznet_lowered.onnx		    old_model
end2end_quartznet_lowered_partitioned.onnx  partitioning_lowering
end2end_quartznet_streamline.onnx	    partitioning_repartition


In [1]:
from finn.util.visualization import showInNetron

showInNetron("/tmp/finn_dev_mirza/end2end_quartznet_lowered_partitioned.onnx")

Serving '/tmp/finn_dev_mirza/end2end_quartznet_lowered_partitioned.onnx' at http://0.0.0.0:8081


In [1]:
from finn.util.visualization import showInNetron

showInNetron("/tmp/finn_dev_mirza/partitioning_repartition/partition_13.onnx")

Serving '/tmp/finn_dev_mirza/partitioning_repartition/partition_13.onnx' at http://0.0.0.0:8081


In [ ]:
from finn.core.modelwrapper import ModelWrapper
from finn.util.basic import get_by_name
from finn.transformation.general import GiveUniqueNodeNames
import finn.transformation.fpgadataflow.convert_to_hls_layers as to_hls


model = ModelWrapper("/tmp/finn_dev_mirza/end2end_quartznet_lowered_partitioned.onnx")
mem_mode = "decoupled"

#partition_dir = build_dir+"/partitioning_hls"
partition_id = 0
for n in model.graph.node:
    if n.op_type=="GenericPartition" and n.name=="GenericPartition_13":
        #inst = GetCustomOp(n)
        prefix = "pt_"+str(partition_id)+"_"

        path_to_partition = get_by_name(n.attribute, "model", "name").s.decode('utf-8')
        model_partition = ModelWrapper(path_to_partition)

        model_partition = model_partition.transform(to_hls.InferConvInpGen(), make_deepcopy=False)
        model_partition = model_partition.transform(to_hls.InferVVAU(), make_deepcopy=False)
        model_partition = model_partition.transform(to_hls.InferQuantizedStreamingFCLayer(mem_mode), make_deepcopy=False)
        model_partition = model_partition.transform(to_hls.InferThresholdingLayer(), make_deepcopy=False)
        model_partition = model_partition.transform(to_hls.InferAddStreamsLayer(), make_deepcopy=False)
        model_partition = model_partition.transform(to_hls.InferDuplicateStreamsLayer(), make_deepcopy=False)

        model_partition = model_partition.transform(GiveUniqueNodeNames(prefix))

        #pathlib.Path(self.partition_dir).mkdir(parents=True, exist_ok=True)
        #partition_path = partition_dir+"/partition_"+str(partition_id)+".onnx"
        model_partition.save(path_to_partition)
        #inst.set_nodeattr("model", partition_path)

        partition_id+=1

model.save(build_dir+"/end2end_quartznet_hls_layers.onnx")